# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
#OLLAMA_API = "http://192.168.0.207:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [24]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from requests.exceptions import RequestException
# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [25]:
# Constants

OLLAMA_API = "http://192.168.0.207:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
ollama_via_openai = OpenAI(base_url='http://192.168.0.207:11434/v1', api_key='ollama')

In [26]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        self.title = "No title found"
        self.text = "No content found"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raises an HTTPError for bad responses
            
            soup = BeautifulSoup(response.content, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            
            # Get text content
            self.text = ' '.join([p.get_text() for p in soup.find_all('p')])
            if not self.text:
                self.text = "No content found"

            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
                
        except requests.exceptions.SSLError:
            raise ConnectionError(f"SSL Certificate verification failed for {url}")
        except requests.exceptions.ConnectionError:
            raise ConnectionError(f"Failed to connect to {url}. Please check if the URL is correct and accessible.")
        except requests.exceptions.Timeout:
            raise ConnectionError(f"Connection timed out while trying to access {url}")
        except requests.exceptions.RequestException as e:
            raise ConnectionError(f"An error occurred while fetching the website: {str(e)}")


In [ ]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [27]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [28]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarise these too. Ignoring text that might be navigation related. \n --- \n"
    user_prompt += website.text
    return user_prompt

In [ ]:
ms=Website("https://technicallysimple.me")
print(user_prompt_for(ms))

In [29]:
# Create a messages list using the same format that we used for OpenAI
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
me=Website("https://technicallysimple.me")
print(messages_for(me))

In [30]:
def summarise(url):
    try:
        aWebsite = Website(url)
        response = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=messages_for(aWebsite)
        )
        return response.choices[0].message.content
    except ConnectionError as e:
        return f"Error: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [31]:
import ollama

def summarise(url):
    try:
        aWebsite = Website(url)
        response = ollama.chat(model=MODEL, messages=messages_for(aWebsite))
        return response['message']['content']
    except ConnectionError as e:
        return f"Error: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

#print(response['message']['content'])

In [32]:
def display_summary(url):
    try:
        summary = summarise(url)
        display(Markdown(summary))
    except Exception as e:
        print(f"Failed to display summary: {str(e)}")

In [ ]:
address=input("Enter URL: ")
summarise(address)

In [35]:
address=input("Enter URL: ")
display_summary(address)

Enter URL:  https://aquasec.com


**Aqua Cloud Native Security**

Aqua is a pioneer in securing containerized cloud native applications from development to production. Their full lifecycle solution prevents attacks by enforcing pre-deployment hygiene and mitigating attacks in real time in production, reducing mean time to repair and overall business risk.

**Recent News**

* **300,000+ Prometheus Servers and Exporters Exposed to DoS Attacks**: Matrix Unleashes A New Widespread DDoS Campaign
* **Aqua Nautilus research team is uniquely focused on the cloud native threat landscape**, with a sole mission to analyze threats targeting cloud native environments and protect customers from those threats.

**Key Features**

* **Cloud Native Application Protection Platform (CNAPP)**: Integrates security from Code to Cloud, combining agent and agentless technology into a single solution.
* **Automate DevSecOps**: Modernize Security with CNDR Cloud Native Detection & Response.
* **Compliance and Auditing**: Serverless Containers & Functions, Hybrid and Multi Cloud, Federal Cloud Native Security.

**Use Cases**

* **Automate DevSecOps**
* **Modernize Security**
* **CNDR Cloud Native Detection & Response**
* **Compliance and Auditing**
* **Serverless Containers & Functions**
* **Hybrid and Multi Cloud**
* **Federal Cloud Native Security**

**Partners**

* **Technology Partners**: Partner With Us
* **Resources**: Aqua Security Research, The Cloud Native Wiki, Kubernetes 101

**About Us**

Aqua is a leading provider of cloud native security solutions, founded in 2015. They protect over 500 of the world's largest enterprises with their enterprise scale solution that doesn’t slow development pipelines.

**Contact Us**

* **Aqua Blog**: Learn more about Aqua Security
* **Get in Touch**: Professional services and support.
* **Careers**: Join the Aqua team